In [ ]:
# monitor_orange_farm.ipynb

from kubernetes import client, config
import pandas as pd
import matplotlib.pyplot as plt
import random
import os

MODE = os.getenv("MODE", "mock")

if MODE == "real":
    # 🌱 Load actual kubeconfig (for local testing)
    config.load_kube_config(config_file="<your_kubeconfig_path>")
    core = client.CoreV1Api()
    print("✅ Loaded real kubeconfig.")
else:
    print("🟠 Running in mock mode — no kubeconfig loaded.")
    core = None  # stub, we won't call real cluster

core = client.CoreV1Api()
metrics = client.CustomObjectsApi()

# 🍊 Get pods
#pods = core.list_pod_for_all_namespaces(watch=False) -- this will be the real pods
#my mock-up pods
pods = [
    {"namespace": "orange-farm", "name": "mandarin-pod-1", "node": "orchard-1", "status": "Running", "app": "juicer"},
    {"namespace": "orange-farm", "name": "mandarin-pod-2", "node": "orchard-1", "status": "Running", "app": "packer"},
    {"namespace": "orange-farm", "name": "mandarin-pod-3", "node": "orchard-2", "status": "Pending", "app": "storer"},
]


pod_data = []
for pod in pods.items:
    pod_data.append({
        "namespace": pod.metadata.namespace,
        "name": pod.metadata.name,
        "node": pod.spec.node_name,
        "status": pod.status.phase,
        "app": pod.metadata.labels.get("app", "unknown")
    })

pods_df = pd.DataFrame(pod_data)


#my mock-up pods cpu and memory
pods_df["cpu(cores)"] = [round(random.uniform(0.05, 0.5), 2) for _ in range(len(pods_df))]
pods_df["memory(Mi)"] = [random.randint(50, 200) for _ in range(len(pods_df))]


# 🧩 Try to fetch metrics if Metrics Server is available
try:
    pod_metrics = metrics.list_cluster_custom_object(
        group="metrics.k8s.io",
        version="v1beta1",
        plural="pods"
    )
    usage_data = []
    for item in pod_metrics["items"]:
        for container in item["containers"]:
            usage_data.append({
                "namespace": item["metadata"]["namespace"],
                "name": item["metadata"]["name"],
                "cpu": container["usage"]["cpu"],
                "memory": container["usage"]["memory"]
            })
    metrics_df = pd.DataFrame(usage_data)
    # Merge pod info with metrics
    pods_df = pods_df.merge(metrics_df, on=["namespace", "name"], how="left")
except Exception as e:
    print("⚠️ Metrics not available — showing pod info only.")
    pods_df["cpu"] = None
    pods_df["memory"] = None

# 📊 Summaries
status_count = pods_df["status"].value_counts()
app_count = pods_df["app"].value_counts()

# 🧺 Plot 1: Pod status
plt.figure(figsize=(6,4))
status_count.plot(kind='bar', color=['orange', 'green', 'red'])
plt.title("Mandarin Pods Status — Orange Farm Cluster")
plt.xlabel("Pod Status")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig("pods_status.png")
plt.show()

# 🍇 Plot 2: Pods per app
plt.figure(figsize=(6,4))
app_count.plot(kind='bar', color='orange')
plt.title("Pods per Application (App Variety in the Orchard)")
plt.xlabel("App Name")
plt.ylabel("Pod Count")
plt.tight_layout()
plt.savefig("pods_per_app.png")
plt.show()

# 🍏 Plot 3: CPU and memory (if available)
if pods_df["cpu"].notna().any():
    pods_df.dropna(subset=["cpu"], inplace=True)
    # Simplify CPU and memory units (rough conversion)
    def parse_cpu(val):
        if val.endswith("n"):
            return int(val[:-1]) / 1e9
        if val.endswith("m"):
            return int(val[:-1]) / 1000
        return int(val)

    def parse_mem(val):
        val = val.lower()
        if val.endswith("ki"):
            return int(val[:-2]) / 1024
        if val.endswith("mi"):
            return int(val[:-2])
        if val.endswith("gi"):
            return int(val[:-2]) * 1024
        return int(val)

    pods_df["cpu_millicores"] = pods_df["cpu"].apply(parse_cpu)
    pods_df["mem_mb"] = pods_df["memory"].apply(parse_mem)

    app_usage = pods_df.groupby("app")[["cpu_millicores", "mem_mb"]].mean()

    app_usage.plot(kind='bar', figsize=(8,4))
    plt.title("Average Resource Use per App (Fertilizer & Water)")
    plt.ylabel("Usage")
    plt.tight_layout()
    plt.savefig("app_resource_usage.png")
    plt.show()